## Here We experimented with the following models
we started our work from extracting keywords 
- Multinomail Naive Bayes 
- Compliment Naive Bayes
- Logistic Regression
- LinearSVC
- Here we are not concerned about keywords to help us in multilabel classification

we are using 2 types of classifications
- One vs Rest
- MultiOutput 

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
import sklearn
import numpy as np 
import warnings
import pandas as pd   
from ast import literal_eval
import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
import pickle
import nltk
import time
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

# Filter out specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

In [2]:
df = pd.read_csv("train.csv", header=0, index_col=0)
df.head()

,Title,Abstract,Categories
Id,,,
9707,Axiomatic Aspects of Default Inference,This paper studies axioms for nonmonotonic con...,['cs.LO']
24198,On extensions of group with infinite conjugacy...,We characterize the group property of being wi...,['math.GR']
35766,An Analysis of Complex-Valued CNNs for RF Data...,Recent deep neural network-based device classi...,"['cs.LG', 'cs.IT', 'eess.SP', 'math.IT']"
14322,On the reconstruction of the drift of a diffus...,The problem of reconstructing the drift of a d...,"['math.PR', 'math.ST', 'stat.TH']"
709,Three classes of propagation rules for GRS and...,"In this paper, we study the Hermitian hulls of...","['cs.IT', 'math.IT']"


In [3]:
target_cols = ["math.AT", "stat.AP", "cs.AR", "math.QA", "q-bio.MN", "eess.AS", "eess.IV", "stat.ME", "econ.GN", "eess.SP", 
               "q-fin.RM", "cs.LG", "cs.CR", "q-bio.BM", "q-fin.GN", "q-fin.MF", "q-fin.PR", "math.CV", "cs.LO", "econ.TH", 
               "math.CO", "cs.AI", "math.AC", "q-bio.CB", "q-fin.CP", "cs.CL", "cs.DC", "math.LO", "math.NT", "cs.SD", 
               "q-fin.TR", "cs.CV", "stat.ML", "q-fin.EC", "econ.EM", "cs.CE", "stat.CO", "math.PR", "q-bio.NC", "math.AP", 
               "cs.OS", "cs.NI", "cs.IT", "cs.PL", "cs.GT", "cs.DM", "math.IT", "cs.SE", "cs.RO", "stat.TH", "cs.DB", 
               "math.ST", "q-bio.GN", "q-fin.PM", "q-bio.TO", "math.GR", "cs.IR"]

In [4]:

nltk.download('stopwords')
nltk.download('wordnet')
# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))


custom_stopwords = [
        "ability", "able", "absolute", "absolutely", "account", "accurate", "achieve", "address",
        "allowing", "also", "analyze", "analyzes", "answer", "application", "approach",
        'any','apply','applying','reapplying','given','papers','paper','about','results','result','real','world',
        'page','article','present','takes','account', 'previous','work','propose','proposes','proposed','simply',
        'simple','demonstrate','demonstrated','demonstrates','realworld','datasets','dataset','provide','important',
        'research','researchers','experiments','experiment','unexpected','discovering','using','recent','collected',
        'solve','columns','existing','traditional','final','consider','presented','provides','automatically','extracting',
        'including','help','helps','explore','illustrate','achieve','better',
        "around", "article", "aspect", "audience", "author", "available", "based", "begin", "best", "better",
        "beyond", "bound", "brief", "called", "capable", "capture", "carefully", "case", "certain", "challenging",
        "compare", "complex", "component", "comprehensive", "concept", "conceptual",
        "conclusion", "condition", "conduct", "conjecture", "consider", "construct", "content", "context", "cost",
        "cross", "crucial", "current", "demonstrate", "derive", "derived", "describe",
        "described", "describes", "detailed", "determine", "developed", "different", "difficult", "directly",
        "discourse", "discuss", "distinguish", "driven", "due", "effect", "effective", "efficient", "efficiently",
        "eight", "element", "emphasis", "end", "enhanced", "evaluate", "even", "example", "experiment", "experimental",
        "explain", "extensive", "family", "feature", "figure", "find", "fine", "finite", "finitely", "first",
        "fit", "five", "found", "four", "framework", "function", "fundamental", "future", "general", "give",
        "given", "good", "grained", "graph", "group", "handed", "high", "higher", "however", "illustrate", "impact",
        "implement", "important", "include", "included", "integrate", "interest", "introduce", "introduced", "introduction",
        "investigate", "issue", "iteration", "known", "large", "last", "leading", "left", "let", "long", "low", "lower", "make",
        "many", "maximal", "may", "method", "methodology", "minimal", "model", "moreover", "multiple",
        "necessary", "need", "needed", "new", "news", "next", "nine", "non", "note", "novel", "number", "numerical",
        "objective", "observables", "observation", 'obtained', "often", "one", "open", "operator", "optimal", "order",
        "outline",
        "outlines", "output", "paper", "papr", "parameter", "part", "particular", "perform",
        "performance", "performed", "performing", "performance", "phase", "point", "possible",
        "potential", "pre", "precisely", "present", "previous", "principle", "problem", "process", "prof",
        "proof", "proper", "property", "propose", "proposed", "proposes", "prove", "provide", "provided",
        "publicly", "publish", "purpose", "quality", "question", "range", "real", "recent", "recently",
        "recommendation", "related", "reliable", "representation", "require", "research", "result", "rev", "review",
        "right", "rigorous", "role", "scale", "scenario", "second", "section", "selection", "series", "serious", "set", "setting",
        "seven", "show", "significant", "significantly", "simulation", "single", "six", "solution", "state",
        "strongly", "structure", "studied", "study", "sufficient", "suggestion", "sum", "synthesize", "system",
        "table", "take", "taken", "task", "technique", "ten", "term", "theorem", "theory", "third",
        "though", "three", "time", "topic", "two", "type", "upper", "use", "used", "using", "utilize", "valid",
        "value", "variable", "variety", "various", "via", "view", "way", "well", "whether", "wide", "widely", "within",
        "without", "work", "world", "written", "year", "zero", "zeroth"]



stop_words.update(custom_stopwords)
lemmatizer = WordNetLemmatizer()


def preprocess_text(text, stop_words, lemmatizer):
    if pd.isna(text):
        return ''

    # Cleaning operations
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)

    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    
    return ' '.join(words)




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df['Title'] = df['Title'].apply(preprocess_text, args=(stop_words, lemmatizer))
df['Abstract'] = df['Abstract'].apply(preprocess_text, args=(stop_words, lemmatizer))
# combine title and abstract into one column and drop original columns
df['Text'] = df['Title'] + ' ' + df['Abstract']
df.head()

,Title,Abstract,Categories,Text
Id,,,,
9707,axiomatic aspect default inference,study axiom nonmonotonic consequence semantics...,['cs.LO'],axiomatic aspect default inference study axiom...
24198,extension infinite conjugacy class,characterize infinite conjugacy class icc conj...,['math.GR'],extension infinite conjugacy class characteriz...
35766,analysis valued cnns data wireless device clas...,deep neural network device classification stud...,"['cs.LG', 'cs.IT', 'eess.SP', 'math.IT']",analysis valued cnns data wireless device clas...
14322,reconstruction drift diffusion transition prob...,reconstructing drift diffusion erre geq transi...,"['math.PR', 'math.ST', 'stat.TH']",reconstruction drift diffusion transition prob...
709,class propagation rule grs egrs code applicati...,hermitian hull extended generalized reed solom...,"['cs.IT', 'math.IT']",class propagation rule grs egrs code applicati...


In [6]:
df['Cat'] = df['Categories'].apply(literal_eval)
df.head()

,Title,Abstract,Categories,Text,Cat
Id,,,,,
9707,axiomatic aspect default inference,study axiom nonmonotonic consequence semantics...,['cs.LO'],axiomatic aspect default inference study axiom...,[cs.LO]
24198,extension infinite conjugacy class,characterize infinite conjugacy class icc conj...,['math.GR'],extension infinite conjugacy class characteriz...,[math.GR]
35766,analysis valued cnns data wireless device clas...,deep neural network device classification stud...,"['cs.LG', 'cs.IT', 'eess.SP', 'math.IT']",analysis valued cnns data wireless device clas...,"[cs.LG, cs.IT, eess.SP, math.IT]"
14322,reconstruction drift diffusion transition prob...,reconstructing drift diffusion erre geq transi...,"['math.PR', 'math.ST', 'stat.TH']",reconstruction drift diffusion transition prob...,"[math.PR, math.ST, stat.TH]"
709,class propagation rule grs egrs code applicati...,hermitian hull extended generalized reed solom...,"['cs.IT', 'math.IT']",class propagation rule grs egrs code applicati...,"[cs.IT, math.IT]"


In [7]:
df['Count'] = df['Cat'].apply(lambda x: len(x))
df.head()

,Title,Abstract,Categories,Text,Cat,Count
Id,,,,,,
9707,axiomatic aspect default inference,study axiom nonmonotonic consequence semantics...,['cs.LO'],axiomatic aspect default inference study axiom...,[cs.LO],1
24198,extension infinite conjugacy class,characterize infinite conjugacy class icc conj...,['math.GR'],extension infinite conjugacy class characteriz...,[math.GR],1
35766,analysis valued cnns data wireless device clas...,deep neural network device classification stud...,"['cs.LG', 'cs.IT', 'eess.SP', 'math.IT']",analysis valued cnns data wireless device clas...,"[cs.LG, cs.IT, eess.SP, math.IT]",4
14322,reconstruction drift diffusion transition prob...,reconstructing drift diffusion erre geq transi...,"['math.PR', 'math.ST', 'stat.TH']",reconstruction drift diffusion transition prob...,"[math.PR, math.ST, stat.TH]",3
709,class propagation rule grs egrs code applicati...,hermitian hull extended generalized reed solom...,"['cs.IT', 'math.IT']",class propagation rule grs egrs code applicati...,"[cs.IT, math.IT]",2


In [8]:
for col in target_cols:
    df[col] = 0

# Update the DataFrame
for index, row in df.iterrows():
    for category in row['Cat']:
        if category in target_cols:
            df.at[index, category] = 1
df.head()

,Title,Abstract,Categories,Text,Cat,Count,math.AT,stat.AP,cs.AR,math.QA,...,cs.SE,cs.RO,stat.TH,cs.DB,math.ST,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
Id,,,,,,,,,,,,,,,,,,,,,
9707,axiomatic aspect default inference,study axiom nonmonotonic consequence semantics...,['cs.LO'],axiomatic aspect default inference study axiom...,[cs.LO],1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24198,extension infinite conjugacy class,characterize infinite conjugacy class icc conj...,['math.GR'],extension infinite conjugacy class characteriz...,[math.GR],1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
35766,analysis valued cnns data wireless device clas...,deep neural network device classification stud...,"['cs.LG', 'cs.IT', 'eess.SP', 'math.IT']",analysis valued cnns data wireless device clas...,"[cs.LG, cs.IT, eess.SP, math.IT]",4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14322,reconstruction drift diffusion transition prob...,reconstructing drift diffusion erre geq transi...,"['math.PR', 'math.ST', 'stat.TH']",reconstruction drift diffusion transition prob...,"[math.PR, math.ST, stat.TH]",3,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
709,class propagation rule grs egrs code applicati...,hermitian hull extended generalized reed solom...,"['cs.IT', 'math.IT']",class propagation rule grs egrs code applicati...,"[cs.IT, math.IT]",2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X = df['Text']
y = df[req_cols]

In [33]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
X_train[:10]

Id
1525     sprout designing expressivity robot fiber embe...
19534    retraction generalized extension computing wor...
22425    improved mass spectrometry compatibility affor...
64986    rgm robust generalist matching finding corresp...
62345    faster ascending auction polymatroid ascending...
4451     quantization poisson group tau connected conne...
7190     accuracy running gsat randomized algorithm dec...
71049    harnessing hard mixed sample decoupled regular...
57731    iterative token evaluation refinement super re...
14039    sentimental education sentiment analysis subje...
Name: Text, dtype: object

In [36]:
y_train[:10]

,math.AT,stat.AP,cs.AR,math.QA,q-bio.MN,eess.AS,eess.IV,stat.ME,econ.GN,eess.SP,...,cs.DM,cs.SE,cs.RO,stat.TH,cs.DB,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
Id,,,,,,,,,,,,,,,,,,,,,
1525,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
19534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
64986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4451,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
def train_eval_pipeline(pipeline: sklearn.pipeline.Pipeline, train_data: tuple,
                        test_data: tuple, name: str) -> str:
    (X_train, y_train) = train_data
    (X_test, y_test) = test_data
    
    start = time.time()
    pipeline.fit(X_train, y_train)
    end = time.time() - start
    
    prediction = pipeline.predict(X_test)
    
    accuracy = accuracy_score(y_test, prediction)
    f1 = f1_score(y_test, prediction, average='micro')  
    
    result_str = 'Accuracy score: {:.2f}%, F1 score: {:.2f}, model trained in {:.2f} secs'.format(accuracy * 100, f1, end)
    
    return result_str

In [38]:
stop_words_list=list(stop_words)

In [27]:
# naive Bayes
ovRest_MNB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

print(train_eval_pipeline(ovRest_MNB_pipeline, (X_train, y_train),
                          (X_test, y_test), "oVRest_MNB"))

Accuracy score: 7.25%, F1 score: 0.17, model trained in 5.94 secs


In [ ]:
# complimentNbB
multOut_CNB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', MultiOutputClassifier(ComplementNB())),])

print(train_eval_pipeline(multOut_CNB_pipeline, (X_train, y_train),
                          (X_test, y_test), "multiCNB"))

Accuracy score: 21.52%, F1 score: 0.43, model trained in 3.94 secs


In [ ]:
multOut_MNB_pipeline= Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', MultiOutputClassifier(MultinomialNB())),])

print(train_eval_pipeline(multOut_MNB_pipeline, (X_train, y_train),
                          (X_test, y_test), "naive_bayes"))

Accuracy score: 6.84%, F1 score: 0.15, model trained in 3.54 secs


In [ ]:
# naive Bayes
ovRest_CNB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', OneVsRestClassifier(ComplementNB())),])

print(train_eval_pipeline(nb_pipeline, (X_train, y_train),
                          (X_test, y_test), "oneVsRestComplimentNB"))

Accuracy score: 21.52%, F1 score: 0.43, model trained in 3.81 secs


In [39]:
# SVM
ovRestSvc_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

print(train_eval_pipeline(ovRestSvc_pipeline, (X_train, y_train), 
                          (X_test, y_test), "svm"))

Accuracy score: 38.55%, F1 score: 0.68, model trained in 16.16 secs


In [ ]:
# SVM
multOutSvc_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', MultiOutputClassifier(LinearSVC(), n_jobs=1)),
            ])

print(train_eval_pipeline(multOutSvc_pipeline, (X_train, y_train), 
                          (X_test, y_test), "svm"))

Accuracy score: 38.30%, F1 score: 0.68, model trained in 17.90 secs


In [ ]:
# Logistic Regression
ovRestLr_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', OneVsRestClassifier(LogisticRegression(), n_jobs=1)),
            ])
print(train_eval_pipeline(ovRestLr_pipeline, (X_train, y_train), 
                          (X_test, y_test), "logistic_regression"))

Accuracy score: 31.54%, F1 score: 0.61, model trained in 31.60 secs


In [31]:
# Logistic Regression
multOutLr_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words_list)),
                ('clf', MultiOutputClassifier(LogisticRegression(), n_jobs=2)),
            ])

print(train_eval_pipeline(multOutLr_pipeline, (X_train, y_train), 
                          (X_test, y_test), "logistic_regression"))

Accuracy score: 31.84%, F1 score: 0.60, model trained in 26.54 secs


In [30]:
test_df=pd.read_csv('test.csv',header=0, index_col=0)
test_df.head(10)
test_df['Title'] = test_df['Title'].apply(preprocess_text, args=(stop_words, lemmatizer))
test_df['Abstract'] = test_df['Abstract'].apply(preprocess_text, args=(stop_words, lemmatizer))
# combine title and abstract into one column and drop original columns
test_df['Text'] = test_df['Title'] + ' ' + test_df['Abstract']
test_df.head()


,Title,Abstract,Text
Id,,,
30332,pricing option intermediate currency,suggest intermediate currency allows price opt...,pricing option intermediate currency suggest i...
50337,multicore processor automobile management,intelligent management managing automobile fun...,multicore processor automobile management inte...
66515,perceptual improvement videoconferencing keyfr...,latest year videoconferencing interpersonal re...,perceptual improvement videoconferencing keyfr...
57464,hundred kilobyte lookup table image super reso...,conventional super resolution scheme heavy con...,hundred kilobyte lookup table image super reso...
43169,sequence labeling actor critic training,neural approach sequence labeling conditional ...,sequence labeling actor critic training neural...


In [ ]:
def make_submission(pipeline): 
    pipeline.fit(X, y)
    
    # Assuming you have a DataFrame named test_df for the test data
    test_X = test_df['Text'].copy()
    
    # Predict on the test data
    prediction = pipeline.predict(test_X)
    
    # Assuming you have a DataFrame named test_df with IDs
    # Set the index of the prediction DataFrame to match the test_df
    predictions_df = pd.DataFrame(prediction, columns=[f'{col}' for col in y.columns])
    predictions_df.index = test_df.index
    
    # Write the predictions to a CSV file
    predictions_df.to_csv('submission.csv')


In [ ]:
make_submission(ovRestSvc_pipeline)

In [ ]:
make_submission(multOutSvc_pipeline)